In [ ]:
import sys
import json
import os
import requests
import random
from datetime import datetime
import time

import pandas as pd
import numpy as np
np.set_printoptions(threshold=np.nan)

import matplotlib.pyplot as plt
%matplotlib notebook


## Run a serving workload

In [ ]:
classes = ['airplane', 'automobile', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck']
positive_class = classes.index('airplane')
negative_class = classes.index('bird')

def load_cifar(cifar_location, cifar_filename = "cifar_train.data", norm=True):
    cifar_path = os.path.join(cifar_location,cifar_filename)
    print("Source file: %s" % cifar_path)
    df = pd.read_csv(cifar_path, sep=",", header=None)
    data = df.values
    print("Number of image files: %d" % len(data))
    y = data[:,0]
    X = data[:,1:]
    Z = X
    if norm:
        mu = np.mean(X,0)
        sigma = np.var(X,0)
        Z = (X - mu) / np.array([np.sqrt(z) if z > 0 else 1. for z in sigma])
    return (Z, y)

def filter_data(X, y):
    X_train, y_train = [], []
    for (example, label) in zip(X, y):
        if label == positive_class:
            X_train.append(example)
            y_train.append(1.0)
        elif label == negative_class:
            X_train.append(example)
            y_train.append(-1.0)
    X_train = np.array(X_train)
    y_train = np.array(y_train)
    return X_train, y_train

            

def cifar_update(host, app, uid, x, y):
    url = "http://%s:1337/%s/update" % (host, app)
    req_json = json.dumps({
        'uid': uid,
        'input': list(x),
        'label': float(y),
        # These updates aren't coming from predictions made by a particular model,
        # so we can ignore the model name and version fields.
        'model_name': 'NA',
        'model_version': 1
        })
    headers = {'Content-type': 'application/json'}
    start = datetime.now()
    r = requests.post(url, headers=headers, data=req_json)
    end = datetime.now()
    latency = (end - start).total_seconds() * 1000.0
    print("'%s', %f ms" % (r.text, latency))

def parse_pred(p):
    splits = p.split(",")
    qid = int(splits[0].strip().split(":")[1])
    pred = float(splits[1].strip().split(":")[1])
    return qid, pred

    
def cifar_prediction(host, app, uid, x):
    url = "http://%s:1337/%s/predict" % (host, app)
    req_json = json.dumps({'uid': uid, 'input': list(x)})
    headers = {'Content-type': 'application/json'}
    # x_str = ", ".join(["%d" % a for a in x])
    start = datetime.now()
    r = requests.post(url, headers=headers, data=req_json)
    end = datetime.now()
    latency = (end - start).total_seconds() * 1000.0
    qid, pred = parse_pred(r.text)
    # TODO TODO TODO: remove check for 0 once selection policy is updated
    if pred == 0.0:
#         print("Warning: Clipper predicted 0.0")
        pred = -1.0
    assert pred == 1.0 or pred == -1.0
#     pred = float(r.text.split(":")[1].strip())
    return (pred, latency)



def send_updates(app, num_updates):
    uid = 0
    for i in range(num_updates):
        example_num = np.random.randint(0, len(test_y))
        cifar_update(host, app, uid, test_x[example_num], float(test_y[example_num]))

def run_iteration(app, uid):
    correct = 0
    false_pos = 0
    false_neg = 0
    latencies = []
    true_pos = 0
    true_neg = 0
    total = 100
    for i in range(total):
        example_num = np.random.randint(0, len(test_y))
        correct_y = float(test_y[example_num])
        pred_y, latency = cifar_prediction(host, app, uid, test_x[example_num])
        if correct_y == pred_y:
            if correct_y == -1:
                true_neg += 1
            elif correct_y == 1:
                true_pos += 1
            correct += 1
        elif correct_y == -1 and pred_y == 1:
            false_pos += 1
        elif correct_y == 1 and pred_y == -1:
            false_neg += 1
        else:
            print "predicted: {p}, correct: {c}".format(p=pred_y, c=correct_y)
        latencies.append(latency)
    total = float(total)    
    return (float(correct)/total,
            float(false_pos)/total,
            float(false_neg)/total,
            float(true_pos)/total,
            float(true_neg)/total,
            np.mean(latencies))

def run_serving_workload(app):
    fig, (ax_acc, ax_lat) = plt.subplots(2,1, sharex=True)
    ax_acc.set_ylabel("accuracy")
    ax_lat.set_xlabel("iterations")
    ax_lat.set_ylabel("latency")
    ax_acc.set_ylim(0, 1.0)
    xs = []
    accs = []
    lats = []
    j = 0
    uid = 0
    while True:
        correct, fp, fn, tp, tn, mean_lat, = run_iteration(app, uid)
        xs.append(j)
        accs.append(correct)
        lats.append(mean_lat)
        j += 1
        ax_acc.set_xlim(0, j + 1)
        ax_lat.set_xlim(0, j + 1)
        
        ax_acc.plot(xs, accs, 'b')
        ax_lat.plot(xs, lats, 'r')
        ax_lat.set_ylim(0, 300)
        fig.canvas.draw()
        print "Accuracy: {cor}, false positives: {fp}, false negatives: {fn}, true positives: {tp}, true negatives: {tn}".format(
            cor=correct, fp=fp, fn=fn, tp=tp, tn=tn)
        print "Mean latency: {lat} ms".format(lat = mean_lat)
        
            
    


In [ ]:
cifar_path = os.path.expanduser("~/model-serving/data/cifar")
x,y = load_cifar(cifar_path, cifar_filename="cifar_test.data")


test_x, test_y = filter_data(x,y)
host = "ec2-54-67-123-237.us-west-1.compute.amazonaws.com"


In [ ]:
# send_updates("rise_demo", 200)
run_serving_workload("rise_demo")

------
## Test code

In [ ]:
example_num = np.random.randint(len(test_x))
print example_num
# my_x = test_x[example_num]

pred_y, latency = cifar_prediction(host, "rise_demo2", 4, test_x[example_num])
print pred_y, latency

In [ ]:
url = "http://localhost:1338/admin/add_app"
candidate_models = [
    {"model_name": "a", "model_version": 1},
    {"model_name": "b", "model_version": 1},
    {"model_name": "c", "model_version": 1},
    {"model_name": "d", "model_version": 1},
    {"model_name": "e", "model_version": 1},
    {"model_name": "f", "model_version": 1},
]

req_json = json.dumps({
    "name": "rise_demo2",
    "candidate_models": candidate_models,
    "input_type": "doubles",
    "output_type": "double",
    "selection_policy": "bandit_policy",
    "latency_slo_micros": 20000
})
headers = {'Content-type': 'application/json'}
r = requests.post(url, headers=headers, data=req_json)
print r.text


In [ ]:
json.dumps(list(test_x[example_num]))